In [31]:
from typing import Tuple, Union, TextIO, List, Dict, Optional, Any, Callable, Sequence, cast
from io import FileIO
from dataclasses import dataclass, replace, field
from functools import reduce
from itertools import chain
import more_itertools as itertools
from enum import IntEnum
from xdsl.dialects import arith, builtin, tensor, linalg, func
from xdsl.frontend.pyast.context import PyASTContext, TypeRegistry
from xdsl.frontend.pyast.code_generation import CodeGeneration, CodeGenerationVisitor
from xdsl import ir
from xdsl.irdl import irdl_op_definition, IRDLOperation, prop_def, result_def, operand_def, var_operand_def, attr_def
from xdsl import passes
import dowhen
import isl

In [32]:
class ParameterizedTypeRegistry(TypeRegistry):
  def __init__(self):
    super().__init__()
    self._generic_mapping: dict[type, type] = {}

  def resolve_attribute(
      self, annotation_name: str, globals: dict[str, Any]
  ) -> ir.TypeAttribute | None:
    """Get an IR type attribute from a string annotation."""
    annotation = cast(
        type,
        eval(annotation_name, globals, None),
    )
    if isinstance(annotation, ir.TypeAttribute):
      return annotation
    return self._mapping.get(annotation, None)

  def get_annotation(self, attribute: ir.TypeAttribute) -> type | None:
    anno = super().get_annotation(attribute)
    if anno is None:
      for key, value in self._generic_mapping.items():
        if value == type(attribute):
          return key
    return anno

  def register_param_type(self, annotation: type,
                          attributeType: type):
    self._generic_mapping[annotation] = attributeType


class MyCodeGenVisitor(CodeGenerationVisitor):
  def parse_op(self, ir_type, func_name, args: tuple):
    ir_type = cast(ir.TypeAttribute, ir_type)
    source_type = self.type_converter.type_registry.get_annotation(ir_type)
    assert source_type
    function_name = f"{source_type.__qualname__}.{func_name}"
    op = self.type_converter.function_registry.resolve_operation(
        module_name=source_type.__module__,
        method_name=function_name,
        args=args,
    )
    assert op
    self.inserter.insert_op(op)

  def visit_Subscript(self, node):
    self.visit(node.slice)
    elts = [self.inserter.get_operand() for i in range(len(node.slice.elts))]
    self.visit(node.value)
    value = self.inserter.get_operand()
    self.parse_op(value.type, '__getitem__', (value, elts[::-1]))

  def visit_Tuple(self, node):
    for elt in node.elts:
      self.visit(elt)

  def visit_Assign(self, node) -> None:
    self.visit(node.targets[0])
    target = self.inserter.get_operand()
    self.visit(node.value)
    value = self.inserter.get_operand()
    self.inserter.insert_op(AssignOp(target, value))


dowhen.when(CodeGeneration.run_with_type_converter, "+9").do(
    lambda type_converter, module, file:
    {"visitor": MyCodeGenVisitor(type_converter, module, file)})


class IterKind(IntEnum):
  Serial = 0
  Distributed = 1
  Tensorize = 2


class UsageKind(IntEnum):
  Input = 0
  Output = 1
  Const = 2


@dataclass(frozen=True)
class IterVar:
  name: str
  lower_bound: int | None
  upper_bound: int | None
  step: int = 1

  @property
  def extent(self) -> int:
    return self.upper_bound - self.lower_bound

  @staticmethod
  def range(name: str, extent: int):
    return IterVar(name, 0, extent, 1)

  @staticmethod
  def symbol(name: str):
    vars = tuple(map(lambda s: IterVar(s, None, None, 1), name.split(' ')))
    return vars[0] if len(vars) == 1 else vars

  def __hash__(self):
    return hash((self.name, self.lower_bound, self.upper_bound, self.step))

  def __str__(self):
    return self.name


@dataclass(frozen=True)
class Buffer:
  name: str
  dims: tuple[int | IterVar, ...]
  dtype: type = float
  sharding: None | isl.map = None
  usage: UsageKind = UsageKind.Input

  @property
  def shape(self) -> tuple[int | IterVar, ...]:
    return tuple(d for d in self.dims)

  @property
  def domain(self) -> isl.set:
    def render(i): return f'0 <= d{i} < {self.dims[i]}' if isinstance(
        self.dims[i], int) else f'{self.dims[i].lower_bound} <= d{i} < {self.dims[i].upper_bound}'
    return isl.set(f"{{ {self.name}[{','.join([f'd{i}' for i in range(len(self.dims))])}] : {' and '.join([render(i) for i in range(len(self.dims))])} }}")

  def __hash__(self):
    return hash((self.name, self.dims, self.dtype, str(self.sharding)))

  @classmethod
  def __class_getitem__(cls, args):
    elem_type = None
    if issubclass(args[0], float):
      elem_type = builtin.f32
    elif issubclass(args[0], int):
      elem_type = builtin.i32
    else:
      raise TypeError(f"Unsupported element type: {args[0]}")
    return builtin.MemRefType(elem_type, [builtin.IntAttr(arg) for arg in args[1]])

  def __setitem__(cls, args):
    raise NotImplementedError()

  def __getitem__(cls, args):
    raise NotImplementedError()

  def __iadd__(self, value):
    raise NotImplementedError()

  def __matmul__(self, value):
    raise NotImplementedError()


@irdl_op_definition
class AccessOp(IRDLOperation):
  name = "buffer.access"

  buffer = operand_def(builtin.MemRefType)
  indices = var_operand_def(builtin.IndexType)
  result = result_def(builtin.Attribute)

  def __init__(
      self,
      buffer: ir.SSAValue,
      indices: Sequence[ir.SSAValue] | ir.SSAValue,
      result_type: ir.Attribute,
  ):
    return super().__init__(operands=[buffer, indices], result_types=[result_type])


@irdl_op_definition
class AssignOp(IRDLOperation):
  name = "tensor.assign"

  target = operand_def(ir.TypeAttribute)
  value = operand_def(ir.TypeAttribute)

  def __init__(
      self,
      target: ir.SSAValue,
      value: ir.SSAValue,
  ):
    return super().__init__(operands=[target, value],
                            result_types=[])


type_registry = ParameterizedTypeRegistry()
type_registry.register_param_type(Buffer, builtin.MemRefType)
ctx = PyASTContext(type_registry)
ctx.register_type(float, builtin.f32)
ctx.register_type(IterVar, builtin.IndexType())
ctx.register_function(Buffer.__getitem__, lambda *args:
                      AccessOp(args[0], args[1], args[0].type.element_type))
ctx.register_function(Buffer.__setitem__, lambda *args:
                      AssignOp(args[0], args[1]))
ctx.register_function(float.__mul__, arith.MulfOp)


@ctx.parse_program
def matmul(A: Buffer[float, [512, 2048]], B: Buffer[float, [2048, 1024]], C: Buffer[float, [512, 1024]], m: IterVar, n: IterVar, k: IterVar) -> Buffer[float, [512, 1024]]:
  C[m, n] = A[m, k] * B[k, n]
  return C


print(matmul.module)

builtin.module {
  func.func @matmul(%A : memref<512x2048xf32>, %B : memref<2048x1024xf32>, %C : memref<512x1024xf32>, %m : index, %n : index, %k : index) -> memref<512x1024xf32> {
    %0 = "buffer.access"(%C, %m, %n) : (memref<512x1024xf32>, index, index) -> f32
    %1 = "buffer.access"(%B, %k, %n) : (memref<2048x1024xf32>, index, index) -> f32
    %2 = "buffer.access"(%A, %m, %k) : (memref<512x2048xf32>, index, index) -> f32
    %3 = arith.mulf %2, %1 : f32
    "tensor.assign"(%0, %3) : (f32, f32) -> ()
    func.return %C : memref<512x1024xf32>
  }
}


In [33]:

@dataclass(frozen=True)
class Mesh:
  dims: tuple[IterVar, ...]

  @property
  def shape(self) -> tuple[int, ...]:
    return tuple(var.extent for var in self.dims)

  @property
  def domain(self) -> isl.set:
    return isl.set(f"{{ Mesh[{','.join(map(str, self.dims))}] : {' and '.join([f'{dim.lower_bound} <= {dim} < {dim.upper_bound}' for dim in self.dims])} }}")


@dataclass(frozen=True)
class Access:
  buffer: Buffer
  relation: isl.map

  def __hash__(self):
    return hash((self.buffer, str(self.relation)))

@dataclass(frozen=True)
class Transfer:
  access: Access
  schedule: isl.map

@dataclass(frozen=True)
class Computation:
  op: str
  domain: isl.set
  schedule: isl.map
  accesses: List[Access]
  iter_vars: tuple[IterVar]
  mesh: Mesh | None = None
  iter_kinds: dict[IterVar, IterKind] = field(default_factory=dict)
  dim_bindings: dict[int, List[Tuple[Access, int]]] = field(default_factory=dict)
  transfers: dict[IterVar, Tuple[Transfer, ...]] = field(default_factory=dict)

  def name(self):
    return self.domain.get_tuple_name()

这里对于index notation的解析仅做简单实现，即iterVar直接参与buffer访问的情况，暂时不支持`A[i+1]`或`B[i+j]`这样复杂的模式，因为在对buffer进行sharding调度时，会需要考虑操作迭代域还是buffer的轴，这是一个困难的问题。

In [34]:
class IndexCollector:
  def __init__(self, iter_set: set[IterVar]):
    self.iter_set: set[IterVar] = iter_set

  def collect(self, node: ir.IRWithUses):
    # todo support complex index pattern
    match node:
      case ir.BlockArgument():
        match node.type:
          case builtin.IndexType():
            iter_var = IterVar.symbol(node.name_hint)
            self.iter_set.add(iter_var)
            return iter_var
      case _:
        raise NotImplementedError("Unsupported index node")


class StmtCollector:
  def __init__(self, stmt_name: str):
    self.stmt_name = stmt_name
    self.iter_set: set[IterVar] = set()
    self.access_dict: dict[AccessOp, (Buffer, list[ir.Operation])] = {}
    self.op: str = None
    self.on_value = False

  def visit(self, node: ir.IRWithUses):
    match node:
      case ir.BlockArgument():
        match node.type:
          case builtin.MemRefType():
            return Buffer(node.name_hint, tuple(dim.data for dim in node.type.shape), node.type.element_type, usage=UsageKind.Input if self.on_value else UsageKind.Output)
          case _:
            return
      case ir.SSAValue() | ir.Operation():
        op = node if isinstance(node, ir.Operation) else node.owner
        match op:
          case AssignOp():
            self.on_value = False
            self.visit(op.target)
            self.on_value = True
            self.visit(op.value)
            self.on_value = False
          case AccessOp():
            b: Buffer = self.visit(op.buffer)
            indices = [IndexCollector(self.iter_set).collect(i) for i in op.indices]
            self.access_dict[op] = (b, indices)
          case arith.MulfOp():
            assert self.op is None
            self.op = op.name
            self.visit(op.lhs)
            self.visit(op.rhs)


class PolyhedronExtractPass(passes.ModulePass):
  name = "polyhedron_analysis"

  computations: list[Computation] = []

  def analysis_stmt(self, op: AssignOp):
    assert len(self.computations) == 0, "not support stmts more than 1"
    stmt_name = f's{len(self.computations)}'
    c = StmtCollector(stmt_name)
    c.visit(op)
    accesses = []
    iters = sorted(c.iter_set, key=lambda i: i.name)
    domain_dims = list(map(lambda x: x.name, iters))
    domain = isl.set(f"{{ {stmt_name}[{','.join(domain_dims)}] }}")
    bounded_iters: dict[str, IterVar] = {}
    dim_bindings: dict[int, List[Tuple[int, int]]] = {}
    for (bf, indices) in c.access_dict.values():
      relation = isl.map(
          f"{{ {stmt_name}[{','.join(domain_dims)}] -> {bf.name}[{','.join(map(lambda x: x.name, indices))}]}}")
      domain = domain.intersect(relation.intersect_range(bf.domain).domain())
      for i, idx in enumerate(indices):
        if idx.name not in bounded_iters:
          bounded_iters[idx.name] = replace(idx, lower_bound=bf.domain.dim_min_val(
              i).num_si(), upper_bound=bf.domain.dim_max_val(i).num_si() + 1)
      access = Access(replace(bf, dims=tuple(
          [bounded_iters[idx.name] for idx in indices])), relation)
      accesses.append(access)
      for i, idx in enumerate(indices):
        j = iters.index(idx)
        binding = dim_bindings.get(j, [])
        binding.append((len(accesses) - 1, i))
        dim_bindings[j] = binding

    comp = Computation(c.op, domain, domain.identity(), accesses,
                       tuple([bounded_iters[v.name] for v in iters]), dim_bindings=dim_bindings)
    self.computations.append(comp)

  def analysis_op(self, op: ir.Operation):
    if isinstance(op, AssignOp):
      self.analysis_stmt(op)

  def apply(self, ctx, op: builtin.ModuleOp) -> dict[str, Computation]:
    for sub in op.walk():
      self.analysis_op(sub)
    return self.computations


s0 = PolyhedronExtractPass().apply(passes.Context(allow_unregistered=True), matmul.module)[0]
s0

Computation(op='arith.mulf', domain=isl.set("{ s0[k, m, n] : 0 <= k <= 2047 and 0 <= m <= 511 and 0 <= n <= 1023 }"), schedule=isl.map("{ s0[k, m, n] -> s0[k' = k, m' = m, n' = n] : 0 <= k <= 2047 and 0 <= m <= 511 and 0 <= n <= 1023 }"), accesses=[Access(buffer=Buffer(name='C', dims=(IterVar(name='m', lower_bound=0, upper_bound=512, step=1), IterVar(name='n', lower_bound=0, upper_bound=1024, step=1)), dtype=Float32Type(), sharding=None, usage=<UsageKind.Output: 1>), relation=isl.map("{ s0[k, m, n] -> C[m, n] }")), Access(buffer=Buffer(name='A', dims=(IterVar(name='m', lower_bound=0, upper_bound=512, step=1), IterVar(name='k', lower_bound=0, upper_bound=2048, step=1)), dtype=Float32Type(), sharding=None, usage=<UsageKind.Input: 0>), relation=isl.map("{ s0[k, m, n] -> A[m, k] }")), Access(buffer=Buffer(name='B', dims=(IterVar(name='k', lower_bound=0, upper_bound=2048, step=1), IterVar(name='n', lower_bound=0, upper_bound=1024, step=1)), dtype=Float32Type(), sharding=None, usage=<UsageKi

In [35]:
def split(self: Computation, parent_var: str, outer_var: str, inner_var: str, factor: int) -> 'Computation':
  """ factor = extend(inner_var) """
  dim_index = self.iter_vars.index(parent_var)
  assert dim_index != -1
  new_iter_vars = tuple([*self.iter_vars[:dim_index],
                         outer_var, inner_var, *self.iter_vars[dim_index + 1:]])

  constraints_str = f' {outer_var} = {parent_var} // {factor} and {inner_var} = {parent_var} - {outer_var} * {factor}'
  mapping_str = f"{{ {self.name()}[{str.join(',', map(str, self.iter_vars))}] -> {self.name()}[{str.join(',', map(str, new_iter_vars))}]: {constraints_str} }}"
  split_map = isl.map(mapping_str)

  new_schedule = self.schedule.apply_range(split_map)
  new_schedule = reduce(lambda sche, p: sche.set_dim_name(
      isl.dim_type.OUT, p[0], str(p[1])), enumerate(new_iter_vars), new_schedule)
  return replace(self, iter_vars=tuple(new_iter_vars), schedule=new_schedule)


Computation.split = split

k, m, n = s0.iter_vars
mo = IterVar.symbol('mo')
mi = IterVar.symbol('mi')

s0_splited = s0.split(m, mo, mi, 8)
s0_splited

Computation(op='arith.mulf', domain=isl.set("{ s0[k, m, n] : 0 <= k <= 2047 and 0 <= m <= 511 and 0 <= n <= 1023 }"), schedule=isl.map("{ s0[k, m, n] -> s0[k' = k, mo, mi = m - 8mo, n' = n] : 0 <= k <= 2047 and 0 <= m <= 511 and 0 <= n <= 1023 and -7 + m <= 8mo <= m }"), accesses=[Access(buffer=Buffer(name='C', dims=(IterVar(name='m', lower_bound=0, upper_bound=512, step=1), IterVar(name='n', lower_bound=0, upper_bound=1024, step=1)), dtype=Float32Type(), sharding=None, usage=<UsageKind.Output: 1>), relation=isl.map("{ s0[k, m, n] -> C[m, n] }")), Access(buffer=Buffer(name='A', dims=(IterVar(name='m', lower_bound=0, upper_bound=512, step=1), IterVar(name='k', lower_bound=0, upper_bound=2048, step=1)), dtype=Float32Type(), sharding=None, usage=<UsageKind.Input: 0>), relation=isl.map("{ s0[k, m, n] -> A[m, k] }")), Access(buffer=Buffer(name='B', dims=(IterVar(name='k', lower_bound=0, upper_bound=2048, step=1), IterVar(name='n', lower_bound=0, upper_bound=1024, step=1)), dtype=Float32Type

In [36]:
def dim_bounds(self: Computation, dim_var: str) -> tuple[isl.val, isl.val]:
  domain = self.schedule.intersect_domain(self.domain).range()
  return (domain.dim_min_val(self.iter_vars.index(dim_var)), domain.dim_max_val(self.iter_vars.index(dim_var)))


Computation.dim_bounds = dim_bounds


def divide(self: Computation, parent_var: str, outer_var: str, inner_var: str, factor: int) -> 'Computation':
  (min_val, max_val) = self.dim_bounds(parent_var)
  return split(self, parent_var, outer_var, inner_var, (max_val.num_si() - min_val.num_si() + 1) // factor)


Computation.divide = divide

no = IterVar.symbol('no')
ni = IterVar.symbol('ni')
s0_divided = s0_splited.divide(n, no, ni, 8)
s0_divided

Computation(op='arith.mulf', domain=isl.set("{ s0[k, m, n] : 0 <= k <= 2047 and 0 <= m <= 511 and 0 <= n <= 1023 }"), schedule=isl.map("{ s0[k, m, n] -> s0[k' = k, mo, mi = m - 8mo, no, ni = n - 128no] : 0 <= k <= 2047 and 0 <= m <= 511 and 0 <= n <= 1023 and -7 + m <= 8mo <= m and -127 + n <= 128no <= n }"), accesses=[Access(buffer=Buffer(name='C', dims=(IterVar(name='m', lower_bound=0, upper_bound=512, step=1), IterVar(name='n', lower_bound=0, upper_bound=1024, step=1)), dtype=Float32Type(), sharding=None, usage=<UsageKind.Output: 1>), relation=isl.map("{ s0[k, m, n] -> C[m, n] }")), Access(buffer=Buffer(name='A', dims=(IterVar(name='m', lower_bound=0, upper_bound=512, step=1), IterVar(name='k', lower_bound=0, upper_bound=2048, step=1)), dtype=Float32Type(), sharding=None, usage=<UsageKind.Input: 0>), relation=isl.map("{ s0[k, m, n] -> A[m, k] }")), Access(buffer=Buffer(name='B', dims=(IterVar(name='k', lower_bound=0, upper_bound=2048, step=1), IterVar(name='n', lower_bound=0, upper_

In [37]:
def reorder(self: Computation, *vars: List[int]):
  assert len(set(vars)) == len(vars)
  g = iter(vars)
  new_iter_vars = [var if var not in vars else next(g) for var in self.iter_vars]
  name = self.domain.get_tuple_name()
  transform = isl.map(
      f"{{ {name}[{','.join(map(str,self.iter_vars))}] -> {name}[{','.join(map(str,new_iter_vars))}] }}")
  new_schedule = self.schedule.apply_range(transform)
  new_schedule = reduce(lambda sche, p: sche.set_dim_name(
      isl.dim_type.OUT, p[0], str(p[1])), enumerate(new_iter_vars), new_schedule)
  return replace(self, iter_vars=tuple(new_iter_vars), schedule=new_schedule)


Computation.reorder = reorder

s0_ordered = s0_divided.reorder(mo, no, mi, ni, k)
s0_ordered

Computation(op='arith.mulf', domain=isl.set("{ s0[k, m, n] : 0 <= k <= 2047 and 0 <= m <= 511 and 0 <= n <= 1023 }"), schedule=isl.map("{ s0[k, m, n] -> s0[mo, no, mi = m - 8mo, ni = n - 128no, k' = k] : 0 <= k <= 2047 and 0 <= m <= 511 and 0 <= n <= 1023 and -7 + m <= 8mo <= m and -127 + n <= 128no <= n }"), accesses=[Access(buffer=Buffer(name='C', dims=(IterVar(name='m', lower_bound=0, upper_bound=512, step=1), IterVar(name='n', lower_bound=0, upper_bound=1024, step=1)), dtype=Float32Type(), sharding=None, usage=<UsageKind.Output: 1>), relation=isl.map("{ s0[k, m, n] -> C[m, n] }")), Access(buffer=Buffer(name='A', dims=(IterVar(name='m', lower_bound=0, upper_bound=512, step=1), IterVar(name='k', lower_bound=0, upper_bound=2048, step=1)), dtype=Float32Type(), sharding=None, usage=<UsageKind.Input: 0>), relation=isl.map("{ s0[k, m, n] -> A[m, k] }")), Access(buffer=Buffer(name='B', dims=(IterVar(name='k', lower_bound=0, upper_bound=2048, step=1), IterVar(name='n', lower_bound=0, upper_

In [38]:
def distribute(self: Computation, parent_vars: List[IterVar], outer_vars: List[IterVar], inner_vars: List[IterVar], mesh: Mesh):
  assert self.mesh is None
  assert len(parent_vars) == len(outer_vars) == len(inner_vars)
  assert all(map(lambda k: k != IterKind.Distributed, self.iter_kinds.values()))
  for i, mesh_dim in enumerate(mesh.dims):
    self = self.divide(parent_vars[i], outer_vars[i], inner_vars[i], mesh_dim.extent)
  self = self.reorder(chain(outer_vars, inner_vars))
  new_iter_kinds = self.iter_kinds.copy()
  new_iter_kinds.update({v: IterKind.Distributed for v in outer_vars})
  return replace(self, iter_kinds=new_iter_kinds, mesh=mesh)


Computation.distribute = distribute

x, y = IterVar.range('x', 8), IterVar.range('y', 8)
mo, no, mi, ni = IterVar.symbol('mo no mi ni')
ko, ki = IterVar.symbol('ko ki')
s0_distributed = s0.distribute([m, n], [mo, no], [mi, ni], Mesh((x, y))). \
    divide(k, ko, ki, x.extent). \
    reorder(mo, no, ko, mi, ni, ki)
s0_distributed

Computation(op='arith.mulf', domain=isl.set("{ s0[k, m, n] : 0 <= k <= 2047 and 0 <= m <= 511 and 0 <= n <= 1023 }"), schedule=isl.map("{ s0[k, m, n] -> s0[mo, no, ko, mi = m - 64mo, ni = n - 128no, ki = k - 256ko] : 0 <= k <= 2047 and 0 <= m <= 511 and 0 <= n <= 1023 and -63 + m <= 64mo <= m and -127 + n <= 128no <= n and -255 + k <= 256ko <= k }"), accesses=[Access(buffer=Buffer(name='C', dims=(IterVar(name='m', lower_bound=0, upper_bound=512, step=1), IterVar(name='n', lower_bound=0, upper_bound=1024, step=1)), dtype=Float32Type(), sharding=None, usage=<UsageKind.Output: 1>), relation=isl.map("{ s0[k, m, n] -> C[m, n] }")), Access(buffer=Buffer(name='A', dims=(IterVar(name='m', lower_bound=0, upper_bound=512, step=1), IterVar(name='k', lower_bound=0, upper_bound=2048, step=1)), dtype=Float32Type(), sharding=None, usage=<UsageKind.Input: 0>), relation=isl.map("{ s0[k, m, n] -> A[m, k] }")), Access(buffer=Buffer(name='B', dims=(IterVar(name='k', lower_bound=0, upper_bound=2048, step=1

In [39]:
@dataclass
class ShardAxis:
  lhs: IterVar
  rhs: Tuple[IterVar]


def var_shard_op(self: IterVar, other: IterVar | List[IterVar]):
  return ShardAxis(self, (other,) if isinstance(other, IterVar) else tuple(other))


IterVar.__matmul__ = var_shard_op


def shard(self: Computation, buffer_name: str, *shard_axes: ShardAxis):
  assert all(map(lambda v: v in self.mesh.dims, chain.from_iterable(
      shard_axis.rhs for shard_axis in shard_axes))), "rhs must be in mesh dimensions"
  access = next(filter(lambda acc: acc.buffer.name == buffer_name, self.accesses))
  assert access
  assert all([shard_axis.lhs in access.buffer.dims for shard_axis in shard_axes]
             ), "lhs must be in buffer dimensions"
  buffer = access.buffer
  assert buffer.sharding == None, "Buffer must not be sharded"

  def bound_fn(v): return f'{v.lower_bound} <= {v} < {v.upper_bound}'
  sharding_map = isl.map(
      f"{{ {buffer.name}[{','.join(map(str,buffer.dims))}] -> {buffer.name}[{','.join(map(str,self.mesh.dims))},{','.join(map(str,buffer.dims))}] : {' and '.join(map(bound_fn, chain(self.mesh.dims, buffer.dims)))} }}")

  local = 'local_'
  space_dims = list(map(str,  chain(self.mesh.dims, buffer.dims)))
  for shard_axis in shard_axes:
    lhs = shard_axis.lhs
    dim_extent = lhs.extent
    for rhs in shard_axis.rhs:
      match rhs:
        # todo support expr.
        case IterVar():
          assert (dim_extent % rhs.extent) == 0, "Dimension can't divide evenly"
          dim_extent = tile = dim_extent // rhs.extent
          constraints = f'{rhs} = {lhs} // {tile} and {local + str(lhs)} = {lhs} - {rhs} * {tile}'
          lhs_space = f"{buffer.name}[{','.join(space_dims)}]"
          rhs_space = f"{buffer.name}[{','.join([local + str(lhs) if dim == str(lhs) else dim for dim in space_dims])}]"
          sharding_map = sharding_map.apply_range(
              isl.map(f'{{ {lhs_space} -> {rhs_space} : {constraints} }}'))
        case _:
          raise NotImplementedError()
  sharding_map = reduce(lambda sche, p: sche.set_dim_name(
      isl.dim_type.OUT, p[0], str(p[1])), enumerate(self.mesh.dims), sharding_map)
  n_access = replace(access, buffer=replace(buffer, sharding=sharding_map))
  return replace(self, accesses=tuple([n_access if o_access.buffer.name == buffer.name else o_access for o_access in self.accesses]))


Computation.shard = shard

s0_sharded = s0_distributed.shard('A', m @ x, k @ y). \
    shard('B', k @ x, n @ y). \
    shard('C', m @ x, n @ y)
s0_sharded

Computation(op='arith.mulf', domain=isl.set("{ s0[k, m, n] : 0 <= k <= 2047 and 0 <= m <= 511 and 0 <= n <= 1023 }"), schedule=isl.map("{ s0[k, m, n] -> s0[mo, no, ko, mi = m - 64mo, ni = n - 128no, ki = k - 256ko] : 0 <= k <= 2047 and 0 <= m <= 511 and 0 <= n <= 1023 and -63 + m <= 64mo <= m and -127 + n <= 128no <= n and -255 + k <= 256ko <= k }"), accesses=(Access(buffer=Buffer(name='C', dims=(IterVar(name='m', lower_bound=0, upper_bound=512, step=1), IterVar(name='n', lower_bound=0, upper_bound=1024, step=1)), dtype=Float32Type(), sharding=isl.map("{ C[m, n] -> C[x, y, m - 64x, local_n = n - 128y] : 0 <= m <= 511 and 0 <= n <= 1023 and 0 <= x <= 7 and -63 + m <= 64x <= m and 0 <= y <= 7 and -127 + n <= 128y <= n }"), usage=<UsageKind.Output: 1>), relation=isl.map("{ s0[k, m, n] -> C[m, n] }")), Access(buffer=Buffer(name='A', dims=(IterVar(name='m', lower_bound=0, upper_bound=512, step=1), IterVar(name='k', lower_bound=0, upper_bound=2048, step=1)), dtype=Float32Type(), sharding=isl

In [40]:
def communicate(self: Computation, buffer_name: str, var: IterVar, rotate_factors: List[IterVar] = []):
  access = next(filter(lambda acc: acc.buffer.name == buffer_name, self.accesses))
  assert access.buffer.sharding
  assert IterKind.Serial == self.iter_kinds.get(var, IterKind.Serial)
  new_transfers = self.transfers.copy()
  transed = new_transfers.get(var, ())
  assert access not in transed

  i = self.iter_vars.index(var)

  access_schedule = None
  if len(rotate_factors) > 0:
    name = self.name()
    extent = self.schedule.range().dim_max_val(i).num_si() + 1
    nvar = str(var) + '_r'
    niter_vars = tuple([*self.iter_vars[:i], nvar, *self.iter_vars[i + 1:]])
    constraint = f"{nvar} = ({' + '.join(map(str, rotate_factors))} + {var}) mod {extent}"
    access_schedule = isl.map(
        f"{{ {name}[{','.join(map(str, self.iter_vars))}] -> {name}[{','.join(map(str, niter_vars))}] : {constraint} }}")
  else:
    access_schedule = self.schedule.range().identity()

  # check validity
  schedule_to_sharding = self.schedule.apply_range(access_schedule).apply_domain(
      access.relation).apply_domain(access.buffer.sharding).reverse()  # schedule -> buffer
  dist_to_shard = schedule_to_sharding.project_out(isl.dim_type.IN, i + 1, len(self.iter_vars) - i - 1). \
      project_out(isl.dim_type.OUT, len(self.mesh.dims),
                  access.buffer.sharding.dim(isl.dim_type.OUT) - len(self.mesh.dims))
  assert dist_to_shard.is_single_valued(), "transfer can't read/write data cross multi nodes."

  new_transfers[var] = (Transfer(access, access_schedule), *transed)
  return replace(self, transfers=new_transfers)


Computation.communicate = communicate

s0_communicated = s0_sharded.communicate('A', ko). \
    communicate('B', ko)

s0_communicated

Computation(op='arith.mulf', domain=isl.set("{ s0[k, m, n] : 0 <= k <= 2047 and 0 <= m <= 511 and 0 <= n <= 1023 }"), schedule=isl.map("{ s0[k, m, n] -> s0[mo, no, ko, mi = m - 64mo, ni = n - 128no, ki = k - 256ko] : 0 <= k <= 2047 and 0 <= m <= 511 and 0 <= n <= 1023 and -63 + m <= 64mo <= m and -127 + n <= 128no <= n and -255 + k <= 256ko <= k }"), accesses=(Access(buffer=Buffer(name='C', dims=(IterVar(name='m', lower_bound=0, upper_bound=512, step=1), IterVar(name='n', lower_bound=0, upper_bound=1024, step=1)), dtype=Float32Type(), sharding=isl.map("{ C[m, n] -> C[x, y, m - 64x, local_n = n - 128y] : 0 <= m <= 511 and 0 <= n <= 1023 and 0 <= x <= 7 and -63 + m <= 64x <= m and 0 <= y <= 7 and -127 + n <= 128y <= n }"), usage=<UsageKind.Output: 1>), relation=isl.map("{ s0[k, m, n] -> C[m, n] }")), Access(buffer=Buffer(name='A', dims=(IterVar(name='m', lower_bound=0, upper_bound=512, step=1), IterVar(name='k', lower_bound=0, upper_bound=2048, step=1)), dtype=Float32Type(), sharding=isl

In [41]:
class OpKind(IntEnum):
  # call(Assign, dest, src)
  Assign = 0
  # call(Access, buffer, *(int | slice))
  Access = 1
  # call(Trans, commPattern, sendbuf, dest, recvbuf, source)
  Trans = 2
  # call(Alloc, name, *dims)
  Alloc = 3
  # call(Rank, *ids)
  Rank = 4
  # call(CommSendrecv)
  CommSendrecv = 5
  # call(CommBroadcast, *commGroups)
  CommBroadcast = 6
  # call(CommShift, *commGroups, direction)
  CommShift = 7
  # call(AssertEqual, a, b)
  AssertEqual = 8
  # call(AugAssign, dest, src)
  AugAssign = 9

  # call(Slice, begin, end)
  Slice = 128
  Add = 129
  Mul = 130
  MatMul = 131


def call_from(build: isl.ast_build, op: OpKind, *args: List[isl.ast_expr | str]):
  assert isinstance(op, OpKind)
  l = isl.ast_expr_list(len(args))
  for i in range(len(args)):
    match args[i]:
      case isl.ast_expr():
        l = l.add(args[i])
      case int() | isl.val():
        l = l.add(isl.ast_expr.from_val(args[i]))
      case str():
        l = l.add(isl.ast_expr.from_id(args[i]))
      case isl.pw_aff():
        l = l.add(build.expr_from(args[i]))
      case _:
        raise ValueError(f"Unsupported argument type: {type(args[i])}")
  return isl.ast_expr.call(isl.ast_expr.from_id(op.name), l)


开始生成计算代码，由于transfer调度引入了新的临时buffer/数据通信，以及tensorize带来的改变。

为正确的构造他

In [42]:
def get_kelly_map(self: Computation, *tps: Tuple[int, int]):
  ndim = self.schedule.dim(isl.dim_type.OUT)
  sche = self.schedule.range().identity()
  d = {p[0] + 1: p[1] for p in tps}
  for i in range(ndim):
    sche = sche.insert_dims(isl.dim_type.OUT, i * 2,
                            1).fix_val(isl.dim_type.OUT, i * 2, d.get(i, 0))
  return sche.set_tuple_name(isl.dim_type.OUT, sche.get_tuple_name(isl.dim_type.IN))


@dataclass(frozen=True, unsafe_hash=True)
class TransferScheduleInfo:
  dim: int
  access_map: isl.map
  access_shard_map: isl.map  # schedule -> buffer
  access_schedule: isl.map  # schedule -> new schedule
  box_hull: isl.fixed_box
  alloc_schedule: isl.map
  trans_schedule: isl.map
  redundancies: Tuple[int, ...]

  @property
  def alloc_name(self):
    return self.alloc_schedule.tuple_name(isl.dim_type.IN)

  @property
  def trans_name(self):
    return self.trans_schedule.tuple_name(isl.dim_type.IN)


@dataclass(frozen=True, unsafe_hash=True)
class ComputationScheduleInfo:
  comp_schedule: isl.map
  assign_kind: OpKind
  op_kind: OpKind

  @property
  def comp_name(self):
    return self.comp_schedule.tuple_name(isl.dim_type.IN)


def get_transfer_schedule_info(self: Computation, transfer: Transfer, dim: int, order: int):
  sched_domain = self.schedule.range()
  ndim = sched_domain.n_dim()
  access_map = self.schedule.apply_domain(transfer.access.relation).reverse()
  access_shard_map = access_map.apply_range(transfer.access.buffer.sharding)
  box_hull = access_map. \
      eliminate(isl.dim_type.IN, dim + 1, ndim - dim - 1). \
      range_simple_fixed_box_hull()

  trans_name = OpKind.Trans.name + transfer.access.buffer.name
  alloc_name = OpKind.Alloc.name + trans_name
  alloc_schedule = get_kelly_map(self, (dim, order)). \
      intersect_domain(sched_domain). \
      project_out(isl.dim_type.IN, dim + 1, ndim - (dim + 1)). \
      set_domain_tuple(alloc_name)
  for drop_dim in range(dim + 1, ndim):
    alloc_schedule = alloc_schedule.fix_si(isl.dim_type.OUT, drop_dim * 2 + 1, 0)
  order += 1

  trans_schedule = get_kelly_map(self, (dim, order)). \
      intersect_domain(sched_domain). \
      set_domain_tuple(trans_name)

  # find dropped dimensions
  redundancies = []
  cons_free_map = access_map.drop_constraints_not_involving_dims(
      isl.dim_type.OUT, 0, len(transfer.access.buffer.dims))
  for i in range(dim, ndim):
    if not cons_free_map.involves_dims(isl.dim_type.IN, i, 1):
      trans_schedule = trans_schedule.fix_si(isl.dim_type.IN, i, 0)
      redundancies.append(i)

  order += 1

  return TransferScheduleInfo(dim, access_map, access_shard_map, transfer.schedule,
                              box_hull, alloc_schedule, trans_schedule,
                              tuple(redundancies))


s0_trans_info_0 = get_transfer_schedule_info(
    s0_communicated, s0_communicated.transfers[ko][0], 2, 0)
s0_trans_info_1 = get_transfer_schedule_info(
    s0_communicated, s0_communicated.transfers[ko][1], 2, 0)
print(s0_trans_info_0)
print(s0_trans_info_1)

TransferScheduleInfo(dim=2, access_map=isl.map("{ s0[mo, no, ko, mi, ni, ki] -> B[256ko + ki, 128no + ni] : mi >= 0 and -64mo <= mi <= 511 - 64mo and mi <= 63 and ni >= 0 and -128no <= ni <= 1023 - 128no and ni <= 127 and ki >= 0 and -256ko <= ki <= 2047 - 256ko and ki <= 255 }"), access_shard_map=isl.map("{ s0[mo, no, ko, mi, ni, ki] -> B[x, y, 256ko + ki - 256x, local_n = 128no + ni - 128y] : mi >= 0 and -64mo <= mi <= 511 - 64mo and mi <= 63 and ni >= 0 and -128no <= ni <= 1023 - 128no and ni <= 127 and ki >= 0 and -256ko <= ki <= 2047 - 256ko and ki <= 255 and 0 <= x <= 7 and -255 + 256ko + ki <= 256x <= 256ko + ki and 0 <= y <= 7 and -127 + 128no + ni <= 128y <= 128no + ni }"), access_schedule=isl.map("{ s0[mo, no, ko, mi, ni, ki] -> s0[mo' = mo, no' = no, ko' = ko, mi' = mi, ni' = ni, ki' = ki] : mi >= 0 and -64mo <= mi <= 511 - 64mo and mi <= 63 and ni >= 0 and -128no <= ni <= 1023 - 128no and ni <= 127 and ki >= 0 and -256ko <= ki <= 2047 - 256ko and ki <= 255 }"), box_hull=isl

由于我们的backend是MPI，没办法直接根据rank id进行数据读写，只能依赖它提供的通信原语。比如当多个rank需要从同一个rank进行取数时， 需要使用broadcast原语才可以正常通信，如果使用sendrecv原语则需要指定rank多次调用。 因此针对通信调度还需要检测其通信模式：

In [43]:
@dataclass(frozen=True, unsafe_hash=True)
class CommPattern:
  def build_call(self, build):
    pass


@dataclass(frozen=True, unsafe_hash=True)
class SendRecv(CommPattern):
  def build_call(self, build):
    return call_from(build, OpKind.CommSendrecv)


@dataclass(frozen=True, unsafe_hash=True)
class Broadcast(CommPattern):
  axes: tuple[int, ...]

  def build_call(self, build):
    return call_from(build, OpKind.CommBroadcast, *self.axes)


@dataclass(frozen=True, unsafe_hash=True)
class Shift(CommPattern):
  axes: tuple[int, ...]
  direction: int

  def build_call(self, build):
    return call_from(build, OpKind.CommShift, *self.axes, self.direction)


def detect_communication_pattern(self: Computation, info: TransferScheduleInfo):
  domain_ndim = info.access_map.dim(isl.dim_type.IN)
  shard_ndim = info.access_shard_map.dim(isl.dim_type.OUT)
  mesh_ndim = len(self.mesh.dims)
  access_shard_map = info.access_shard_map.apply_domain(info.access_schedule)
  access_src_rank = access_shard_map. \
      project_out(isl.dim_type.OUT, mesh_ndim, shard_ndim - mesh_ndim).\
      project_out(isl.dim_type.IN, info.dim + 1, domain_ndim - info.dim - 1)
  access_src_pma = access_src_rank.as_pw_multi_aff()
  access_next_src_pma = isl.pw_multi_aff.identity_on_domain(
      access_src_pma.domain_space()).add_constant(isl.multi_val(f"{{ [{','.join(['0'] * info.dim + ['1'])}] }}"))
  src_rank_deltas = access_src_pma.pullback(access_next_src_pma).sub(access_src_pma).coalesce()
  comm_patterns = []
  for i in range(mesh_ndim):
    pa = src_rank_deltas.at(i)
    if pa.is_cst():
      match pa.max_val().num_si():
        case 0:  # not involved
          comm_patterns.append(SendRecv())
        case 1:  # identity with time.
          if not access_src_rank.is_bijective():  # detect broadcast
            unbounded = access_src_rank.drop_constraints_not_involving_dims(
                isl.dim_type.OUT, 0, mesh_ndim)
            if not unbounded.involves_dims(isl.dim_type.IN, i, 1):
              comm_patterns.append(Broadcast((i,)))
            else:
              comm_patterns.append(SendRecv())
          else:
            comm_patterns.append(SendRecv())
    else:
      points = []
      pa.as_map().range().foreach_point(lambda x: points.append(isl.set(x)))
      points = reduce(lambda acc, x: acc.union(x), points, isl.set.empty(
          isl.space.unit().add_dims(isl.dim_type.SET, 1)))
      extent = self.schedule.range().dim_max_val(info.dim).num_si()
      cw_set = isl.set(f'{{[1]; [-{extent}]}}')
      ccw_set = isl.set(f'{{[-1]; [{extent}]}}')
      if points.is_equal(cw_set):
        comm_patterns.append(Shift((i,), 1))
      elif points.is_equal(ccw_set):
        comm_patterns.append(Shift((i,), -1))

  special = sum([isinstance(p, (Broadcast, Shift)) for p in comm_patterns])
  assert special <= 1
  return SendRecv() if special == 0 else next(filter(lambda p: not isinstance(p, SendRecv), comm_patterns))


print(detect_communication_pattern(s0_communicated, s0_trans_info_0))
print(detect_communication_pattern(s0_communicated, s0_trans_info_1))

Broadcast(axes=(0,))
Broadcast(axes=(1,))


In [44]:
@dataclass(frozen=True, unsafe_hash=True)
class ScheduleInfo:
  trans_infos: Tuple[TransferScheduleInfo]
  comp_infos: Tuple[ComputationScheduleInfo]


def get_schedule_info(self: Computation):
  transfer_sche_infos = []
  used_orders = []
  for (var, accesses) in self.transfers.items():
    dim = self.iter_vars.index(var)
    order = 0
    for access in accesses:
      transfer_sche_infos.append(get_transfer_schedule_info(self, access, dim, order))
      order += 2
      used_orders.append((dim, order))
  in_iters = set(itertools.flatten(
      [acc.buffer.dims for acc in self.accesses if acc.buffer.usage is UsageKind.Input]))
  out_iters = set(itertools.flatten(
      [acc.buffer.dims for acc in self.accesses if acc.buffer.usage is UsageKind.Output]))

  op_kind = None
  match self.op:
    case OpKind():
      op_kind = self.op
    case 'arith.mulf':
      op_kind = OpKind.Mul
    case _:
      raise NotImplementedError()

  comp_schedule = ComputationScheduleInfo(get_kelly_map(
      self, *used_orders).intersect_domain(self.schedule.range()),
      OpKind.AugAssign if len(in_iters) > len(out_iters) else OpKind.Assign,
      op_kind)
  return ScheduleInfo(tuple(transfer_sche_infos), (comp_schedule,))


s0_schedule_info = get_schedule_info(s0_communicated)
s0_schedule_info

ScheduleInfo(trans_infos=(TransferScheduleInfo(dim=2, access_map=isl.map("{ s0[mo, no, ko, mi, ni, ki] -> B[256ko + ki, 128no + ni] : mi >= 0 and -64mo <= mi <= 511 - 64mo and mi <= 63 and ni >= 0 and -128no <= ni <= 1023 - 128no and ni <= 127 and ki >= 0 and -256ko <= ki <= 2047 - 256ko and ki <= 255 }"), access_shard_map=isl.map("{ s0[mo, no, ko, mi, ni, ki] -> B[x, y, 256ko + ki - 256x, local_n = 128no + ni - 128y] : mi >= 0 and -64mo <= mi <= 511 - 64mo and mi <= 63 and ni >= 0 and -128no <= ni <= 1023 - 128no and ni <= 127 and ki >= 0 and -256ko <= ki <= 2047 - 256ko and ki <= 255 and 0 <= x <= 7 and -255 + 256ko + ki <= 256x <= 256ko + ki and 0 <= y <= 7 and -127 + 128no + ni <= 128y <= 128no + ni }"), access_schedule=isl.map("{ s0[mo, no, ko, mi, ni, ki] -> s0[mo' = mo, no' = no, ko' = ko, mi' = mi, ni' = ni, ki' = ki] : mi >= 0 and -64mo <= mi <= 511 - 64mo and mi <= 63 and ni >= 0 and -128no <= ni <= 1023 - 128no and ni <= 127 and ki >= 0 and -256ko <= ki <= 2047 - 256ko and k

In [45]:
def lower_computation(self: Computation):
  schedule_info = get_schedule_info(self)
  # process tensorized
  sched_domain = self.schedule.range()
  sched_domain_min = self.schedule.range()
  sched_domain_max = self.schedule.range()
  tensorized_dims = []
  for i, v in enumerate(self.iter_vars):
    if self.iter_kinds.get(v) is IterKind.Tensorize:
      sched_domain_min = sched_domain_min.fix_val(isl.dim_type.SET, i, sched_domain.dim_min_val(i))
      sched_domain_max = sched_domain_max.fix_val(isl.dim_type.SET, i, sched_domain.dim_max_val(i))
      tensorized_dims.append(i)

  def fix_dims(sche: isl.map):
    for d in tensorized_dims:
      sche = sche.fix_si(isl.dim_type.OUT, (2 * d) + 1, 0)
    return sche

  def drop_dims(sche: isl.map | isl.aff, redundancies: Tuple[int] = ()):
    dims = list(set([*redundancies, *tensorized_dims]))
    dims.sort()
    j = 0
    for i in range(len(dims)):
      if isinstance(sche, isl.map):
        sche = sche.project_out(isl.dim_type.IN, dims[i] - j, 1)
      elif isinstance(sche, isl.pw_aff):
        sche = sche.drop_dims(isl.dim_type.IN, dims[i] - j, 1)
      else:
        raise NotImplementedError
      j += 1
    return sche

  def get_box(map: isl.map) -> isl.multi_val:
    min = drop_dims(map.intersect_domain(sched_domain_min))
    max = drop_dims(map.intersect_domain(sched_domain_max))
    diff = max.as_pw_multi_aff().sub(min.as_pw_multi_aff())
    assert diff.is_cst()
    return diff.max_multi_val()

  full_sche_map = isl.union_map.empty()
  for info in schedule_info.comp_infos:
    full_sche_map = full_sche_map.union(fix_dims(info.comp_schedule))
  for info in schedule_info.trans_infos:
    full_sche_map = full_sche_map.union(fix_dims(info.alloc_schedule))
    full_sche_map = full_sche_map.union(fix_dims(info.trans_schedule))
  alloc_info_map = {info.alloc_name: info for info in schedule_info.trans_infos}
  trans_info_map = {info.trans_name: info for info in schedule_info.trans_infos}
  comp_info_map = {info.comp_name: info for info in schedule_info.comp_infos}

  def at_each_domain(node: isl.ast_node_user, build: isl.ast_build) -> isl.ast_node:
    origin_expr = node.expr()
    if not isinstance(origin_expr, isl.ast_expr_op_call):
      return node

    call_id: isl.ast_expr_id = origin_expr.op_arg(0)
    call_id_name = call_id.id().name()
    # alloc
    if call_id_name in alloc_info_map:
      info = alloc_info_map[call_id_name]
      box_shape = info.box_hull.get_size()
      rank = box_shape.size()
      alloc = call_from(build, OpKind.Alloc, info.trans_name,
                        *[box_shape.at(i) for i in range(rank)])
      return isl.ast_node_user(alloc)

    # trans
    if call_id_name in trans_info_map:
      info = trans_info_map[call_id_name]
      pattern = detect_communication_pattern(self, info)
      select_ranks = list(range(len(self.mesh.dims)))
      match pattern:
        case Broadcast():
          select_ranks = list(filter(lambda i: i in pattern.axes, select_ranks))

      def drop_dims2(x): return drop_dims(x, info.redundancies)

      src_shard_pma = info.access_shard_map.as_pw_multi_aff()
      access_sche_pma = info.access_schedule.as_pw_multi_aff()
      src_shard_pma = src_shard_pma.pullback(access_sche_pma)
      src_rank = call_from(build, OpKind.Rank, *[drop_dims2(src_shard_pma.at(i))
                           for i in select_ranks])

      dest_rank_pma = info.access_map.domain().identity().as_pw_multi_aff()
      dest_rank_pma = dest_rank_pma.pullback(access_sche_pma)
      dest_rank = call_from(build, OpKind.Rank, *
                            [drop_dims2(dest_rank_pma.at(i)) for i in select_ranks])

      src_tensor_box = get_box(info.access_shard_map)
      src_slice = call_from(build, OpKind.Access, info.access_shard_map.tuple_name(isl.dim_type.OUT),
                            *[call_from(build, OpKind.Slice, drop_dims2(src_shard_pma.at(i)),
                                        drop_dims2(src_shard_pma.at(i)).add_constant(src_tensor_box.at(i)).add_constant(1))
                              for i in range(len(self.mesh.dims), src_shard_pma.dim(isl.dim_type.OUT))])

      dest_start_pma = info.box_hull.get_offset().as_pw_multi_aff()
      dest_origin_pma = info.access_map.as_pw_multi_aff()
      dest_tensor_box = get_box(info.access_map)
      dest_pma = dest_origin_pma.sub(dest_start_pma)
      dest_pma = dest_pma.pullback(access_sche_pma)
      dest_slice = call_from(build, OpKind.Access, info.trans_name,
                             *[call_from(build, OpKind.Slice, drop_dims2(dest_pma.at(i)),
                                         drop_dims2(dest_pma.at(i)).add_constant(dest_tensor_box.at(i)).add_constant(1))
                               for i in range(dest_pma.size())])

      trans = call_from(build, OpKind.Trans, pattern.build_call(build),
                        src_slice, src_rank, dest_slice, dest_rank)
      return isl.ast_node_user(trans)
    if call_id_name in comp_info_map:
      info = comp_info_map[call_id_name]
      access_exprs = []
      for access in self.accesses:
        trans_name = OpKind.Trans.name + access.buffer.name
        if trans_name in trans_info_map:
          trans_info = trans_info_map[trans_name]
          access_sche_pma = trans_info.access_schedule.as_pw_multi_aff()
          dest_start_pma = trans_info.box_hull.get_offset().as_pw_multi_aff()
          dest_origin_pma = trans_info.access_map.as_pw_multi_aff()
          dest_tensor_box = get_box(trans_info.access_map)
          dest_pma = dest_origin_pma.sub(dest_start_pma)
          dest_pma = dest_pma.pullback(access_sche_pma)
          access_exprs.append(call_from(build, OpKind.Access, trans_name, *
                              [call_from(build, OpKind.Slice, drop_dims(dest_pma.at(i)), drop_dims(dest_pma.at(i)).add_constant(dest_tensor_box.at(i)).add_constant(1))
                               for i in range(dest_pma.size())]))
        else:
          access_shard_map = self.schedule.reverse().apply_range(
              access.relation).apply_range(access.buffer.sharding)
          dest_shard_pma = access_shard_map.as_pw_multi_aff()
          dest_tensor_box = get_box(access_shard_map)
          dest_slice = call_from(build, OpKind.Access, access.buffer.name, *[
              call_from(build, OpKind.Slice, drop_dims(dest_shard_pma.at(i)),
                        drop_dims(dest_shard_pma.at(i)).add_constant(dest_tensor_box.at(i)).add_constant(1))
              for i in range(len(self.mesh.dims), dest_shard_pma.dim(isl.dim_type.OUT))])
          access_exprs.append(dest_slice)
      call_expr = call_from(build, info.assign_kind,
                            access_exprs[0],
                            call_from(build, info.op_kind, access_exprs[1], access_exprs[2]))
      return isl.ast_node_user(call_expr)
    return node

  builtin_iters = list(map(str, self.mesh.dims))

  def at_each_for(node: isl.ast_node_for, build: isl.ast_build) -> isl.ast_node:
    it = node.get_iterator()
    if isinstance(it, isl.ast_expr_id) and it.id().name() in builtin_iters:
      node = node.set_annotation(IterKind.Distributed.name)
    return node

  ast_build = isl.ast_build()
  ast_build = ast_build.set_at_each_domain(at_each_domain)
  ast_build = ast_build.set_after_each_for(at_each_for)
  iter_ids = []
  ndim = len(self.iter_vars)
  comp_schedule = schedule_info.comp_infos[0].comp_schedule
  iter_kinds = {k.name: v for (k, v) in self.iter_kinds.items()}
  for i in range(ndim):
    iter_ids.append(f'c{i}')
    name = comp_schedule.dim_name(isl.dim_type.OUT, 2 * i + 1)
    if iter_kinds.get(name) is IterKind.Distributed:
      name = str(self.mesh.dims[i])
    iter_ids.append(name)
  ast_build = ast_build.set_iterators('(' + ','.join(iter_ids) + ')')
  ast_node = ast_build.node_from_schedule_map(full_sche_map)
  return ast_node


with open('tmp/1.py', 'w') as f:
  ast_computation = lower_computation(s0_communicated)
  printer = isl.printer.to_file(f)
  print_options = isl.ast_print_options.alloc()
  printer.set_output_format(isl.format.C)
  printer = ast_computation.print(printer, print_options)
  printer.flush()

with open('tmp/1.py', 'r') as f:
  print(f.read())

for (int x = 0; x <= 7; x += 1)
  for (int y = 0; y <= 7; y += 1)
    for (int ko = 0; ko <= 7; ko += 1) {
      Alloc(TransB, 256, 128);
      for (int ni = 0; ni <= 127; ni += 1)
        for (int ki = 0; ki <= 255; ki += 1)
          Trans(CommBroadcast(0), Access(B, Slice(ki, ki + 1), Slice(ni, ni + 1)), Rank(ko), Access(TransB, Slice(ki, ki + 1), Slice(ni, ni + 1)), Rank(x));
      Alloc(TransA, 64, 256);
      for (int mi = 0; mi <= 63; mi += 1)
        for (int ki = 0; ki <= 255; ki += 1)
          Trans(CommBroadcast(1), Access(A, Slice(mi, mi + 1), Slice(ki, ki + 1)), Rank(ko), Access(TransA, Slice(mi, mi + 1), Slice(ki, ki + 1)), Rank(y));
      for (int mi = 0; mi <= 63; mi += 1)
        for (int ni = 0; ni <= 127; ni += 1)
          for (int ki = 0; ki <= 255; ki += 1)
            AugAssign(Access(C, Slice(mi, mi + 1), Slice(ni, ni + 1)), Mul(Access(TransA, Slice(mi, mi + 1), Slice(ki, ki + 1)), Access(TransB, Slice(ki, ki + 1), Slice(ni, ni + 1))));
    }



In [ ]:
def tensorize(self: Computation, vars: List[IterVar], new_op: OpKind = None):
  new_kinds = self.iter_kinds.copy()
  for var in vars:
    assert new_kinds.get(var, None) in (None, IterKind.Serial)
    new_kinds[var] = IterKind.Tensorize
  if new_op:
    self = replace(self, op=new_op)
  return replace(self, iter_kinds=new_kinds)


Computation.tensorize = tensorize

s0_tensorized = s0_communicated.tensorize([mi, ni, ki], OpKind.MatMul)
s0_tensorized

Computation(op=<OpKind.MatMul: 131>, domain=isl.set("{ s0[k, m, n] : 0 <= k <= 2047 and 0 <= m <= 511 and 0 <= n <= 1023 }"), schedule=isl.map("{ s0[k, m, n] -> s0[mo, no, ko, mi = m - 64mo, ni = n - 128no, ki = k - 256ko] : 0 <= k <= 2047 and 0 <= m <= 511 and 0 <= n <= 1023 and -63 + m <= 64mo <= m and -127 + n <= 128no <= n and -255 + k <= 256ko <= k }"), accesses=(Access(buffer=Buffer(name='C', dims=(IterVar(name='m', lower_bound=0, upper_bound=512, step=1), IterVar(name='n', lower_bound=0, upper_bound=1024, step=1)), dtype=Float32Type(), sharding=isl.map("{ C[m, n] -> C[x, y, m - 64x, local_n = n - 128y] : 0 <= m <= 511 and 0 <= n <= 1023 and 0 <= x <= 7 and -63 + m <= 64x <= m and 0 <= y <= 7 and -127 + n <= 128y <= n }"), usage=<UsageKind.Output: 1>), relation=isl.map("{ s0[k, m, n] -> C[m, n] }")), Access(buffer=Buffer(name='A', dims=(IterVar(name='m', lower_bound=0, upper_bound=512, step=1), IterVar(name='k', lower_bound=0, upper_bound=2048, step=1)), dtype=Float32Type(), shar

In [ ]:
with open('tmp/1.py', 'w') as f:
  ast_computation = lower_computation(s0_tensorized)
  printer = isl.printer.to_file(f)
  print_options = isl.ast_print_options.alloc()
  printer.set_output_format(isl.format.C)
  printer = ast_computation.print(printer, print_options)
  printer.flush()

with open('tmp/1.py', 'r') as f:
  print(f.read())

for (int x = 0; x <= 7; x += 1)
  for (int y = 0; y <= 7; y += 1)
    for (int ko = 0; ko <= 7; ko += 1) {
      Alloc(TransB, 256, 128);
      Trans(CommBroadcast(0), Access(B, Slice(0, 256), Slice(0, 128)), Rank(ko), Access(TransB, Slice(0, 256), Slice(0, 128)), Rank(x));
      Alloc(TransA, 64, 256);
      Trans(CommBroadcast(1), Access(A, Slice(0, 64), Slice(0, 256)), Rank(ko), Access(TransA, Slice(0, 64), Slice(0, 256)), Rank(y));
      AugAssign(Access(C, Slice(0, 64), Slice(0, 128)), MatMul(Access(TransA, Slice(0, 64), Slice(0, 256)), Access(TransB, Slice(0, 256), Slice(0, 128))));
    }



In [ ]:
def lower_shard(self: Computation, access: Access) -> Tuple[isl.ast_build, isl.ast_build]:
  sharding = access.buffer.sharding
  assert sharding
  builtin_iters = list(map(str, self.mesh.dims))

  access_global_pma = sharding.reverse().set_tuple_id(
      isl.dim_type.OUT, 'Global' + access.buffer.name).as_pw_multi_aff()
  access_global_domain = sharding.reverse().domain()
  buffer_local_shape = access_global_domain.project_out(
      isl.dim_type.SET, 0, len(self.mesh.dims)).simple_fixed_box_hull().size()
  access_local_map = sharding.reverse().domain().identity().project_out(
      isl.dim_type.OUT, 0, len(self.mesh.dims)).set_range_tuple(access.buffer.name)
  access_local_pma = access_local_map.as_pw_multi_aff()

  ast_build = isl.ast_build()

  def at_each_domain(node: isl.ast_node_user, build: isl.ast_build) -> isl.ast_node:
    access_global = call_from(build, OpKind.Access, access_global_pma.tuple_name(
        isl.dim_type.OUT), *[access_global_pma.at(i) for i in range(access_global_pma.size())])
    access_local = call_from(build, OpKind.Access, access_local_pma.tuple_name(
        isl.dim_type.OUT), *[access_local_pma.at(i) for i in range(access_local_pma.size())])

    call = call_from(build, OpKind.Assign if access.buffer.usage ==
                     UsageKind.Input else OpKind.AssertEqual, access_local, access_global)
    return isl.ast_node_user(call)

  def at_each_for(node: isl.ast_node_for, build: isl.ast_build) -> isl.ast_node:
    it = node.get_iterator()
    if isinstance(it, isl.ast_expr_id) and it.id().name() in builtin_iters:
      node = node.set_annotation(IterKind.Distributed.name)
    return node

  ast_build = ast_build.set_at_each_domain(at_each_domain)
  ast_build = ast_build.set_after_each_for(at_each_for)
  ast_build = ast_build.set_iterators(
      '(' + ', '.join(map(str, chain(self.mesh.dims, access.buffer.dims))) + ')')
  ast_node = ast_build.node_from_schedule_map(sharding.range().identity())
  alloc = isl.ast_node_user(call_from(ast_build, OpKind.Alloc, access.buffer.name, *
                                      [buffer_local_shape.at(i) for i in range(buffer_local_shape.size())]))
  return (alloc, ast_node)

with open('tmp/1.py', 'w') as f:
  ast_shard_buffer_0 = lower_shard(s0_communicated, s0_communicated.accesses[0])
  ast_shard_buffer_1 = lower_shard(s0_communicated, s0_communicated.accesses[1])
  ast_shard_buffer_2 = lower_shard(s0_communicated, s0_communicated.accesses[2])
  printer = isl.printer.to_file(f)
  print_options = isl.ast_print_options.alloc()
  printer.set_output_format(isl.format.C)
  printer = ast_shard_buffer_0[0].print(printer, print_options)
  printer = ast_shard_buffer_0[1].print(printer, print_options)
  printer = ast_shard_buffer_1[0].print(printer, print_options)
  printer = ast_shard_buffer_1[1].print(printer, print_options)
  printer = ast_shard_buffer_2[0].print(printer, print_options)
  printer = ast_shard_buffer_2[1].print(printer, print_options)
  printer.flush()
  
with open('tmp/1.py', 'r') as f:
  print(f.read())

Alloc(C, 64, 128);
for (int x = 0; x <= 7; x += 1)
  for (int y = 0; y <= 7; y += 1)
    for (int m = 0; m <= 63; m += 1)
      for (int n = 0; n <= 127; n += 1)
        AssertEqual(Access(C, m, n), Access(GlobalC, 64 * x + m, 128 * y + n));
Alloc(A, 64, 256);
for (int x = 0; x <= 7; x += 1)
  for (int y = 0; y <= 7; y += 1)
    for (int m = 0; m <= 63; m += 1)
      for (int k = 0; k <= 255; k += 1)
        Assign(Access(A, m, k), Access(GlobalA, 64 * x + m, 256 * y + k));
Alloc(B, 256, 128);
for (int x = 0; x <= 7; x += 1)
  for (int y = 0; y <= 7; y += 1)
    for (int k = 0; k <= 255; k += 1)
      for (int n = 0; n <= 127; n += 1)
        Assign(Access(B, k, n), Access(GlobalB, 256 * x + k, 128 * y + n));



In [ ]:

def print_python_style_item(printer: isl.printer, item: isl.ast_expr | str):
  match item:
    case str():
      printer.print_str(item)
    case isl.ast_expr():
      expr = item
      match expr:
        case isl.ast_expr_op_call():
          op_name = expr.get_op_arg(0).id().name()
          match op_name:
            case OpKind.Access.name:
              print_python_style_item(printer, expr.get_op_arg(1))  # buffer name
              printer.print_str("[")
              for i in range(2, expr.op_n_arg()):
                print_python_style_item(printer, expr.get_op_arg(i))
                printer.print_str(", "[i - expr.op_n_arg():-1])
              printer.print_str("]")
            case OpKind.Rank.name:
              printer.print_str("[")
              for i in range(1, expr.op_n_arg()):
                print_python_style_item(printer, expr.get_op_arg(i))
                printer.print_str(", "[i - expr.op_n_arg():-1])
              printer.print_str("]")
            case OpKind.Assign.name:
              print_python_style_items(printer, expr.get_op_arg(1), " = ", expr.get_op_arg(2))
            case OpKind.AugAssign.name:
              print_python_style_items(printer, expr.get_op_arg(1), " += ", expr.get_op_arg(2))
            case OpKind.Slice.name:
              print_python_style_items(printer, expr.get_op_arg(1), ":", expr.get_op_arg(2))
              if expr.op_n_arg() > 3:
                printer.print_str(":")
                print_python_style_item(printer, expr.get_op_arg(3))
            case OpKind.Alloc.name:
              print_python_style_item(printer, expr.get_op_arg(1))
              printer.print_str(" = np.zeros([")
              for i in range(2, expr.op_n_arg()):
                print_python_style_item(printer, expr.get_op_arg(i))
                printer.print_str(", "[i - expr.op_n_arg():-1])
              printer.print_str("])")
            case OpKind.AssertEqual.name:
              print_python_style_items(printer, "assert np.allclose(", expr.get_op_arg(1), ", ", expr.get_op_arg(2), ")")
            case OpKind.Trans.name:
              printer.print_str(op_name)
              printer.print_str('(')
              for i in range(1, expr.op_n_arg()):
                print_python_style_item(printer, expr.get_op_arg(i))
                printer.print_str(", "[i - expr.op_n_arg():-1])
              printer.print_str(')')
            case OpKind.Mul.name:
              print_python_style_items(printer, expr.get_op_arg(1), " * ", expr.get_op_arg(2))
            case OpKind.MatMul.name:
              print_python_style_items(printer, expr.get_op_arg(1), " @ ", expr.get_op_arg(2))
            case OpKind.CommShift.name:
              print_python_style_items(printer, expr.get_op_arg(0), "(")
              print_python_style_item(printer, '(')
              for i in range(1, expr.op_n_arg() - 1):
                print_python_style_items(printer, expr.get_op_arg(i), ',')
              print_python_style_item(printer, '),')
              print_python_style_item(printer, expr.get_op_arg(expr.op_n_arg() - 1))
              print_python_style_item(printer, ')')
            case _:
              printer.print_ast_expr(expr)
        case _:
          printer.print_ast_expr(expr)
    case _:
      raise NotImplementedError()


def print_python_style_items(printer: isl.printer, *items: List[isl.ast_expr | str]):
  for item in items:
    print_python_style_item(printer, item)


def print_python_style_user(printer: isl.printer, options: isl.ast_print_options, node: isl.ast_node):
  expr: isl.ast_expr = node.get_expr()
  printer.start_line()
  print_python_style_item(printer, expr)
  printer.end_line()
  return printer


def print_python_style_for(printer: isl.printer, options: isl.ast_print_options, node: isl.ast_node_for):
  (it, init, cond, inc) = node.get_iterator(), node.get_init(), node.get_cond(), node.get_inc()
  omit = False
  try:
    anno = node.annotation()
    if anno.name() == IterKind.Distributed.name:
      omit = True
  except:
    pass

  if not omit:
    printer.start_line()
    print_python_style_items(printer, "for ", it, " in range(", init,
                             ", ", cond.get_arg(1), " + 1", ", ", inc, "):\n")
    printer = printer.indent(4)
  printer = node.get_body().print(printer, options)
  if not omit:
    printer = printer.indent(-4)
  return printer


def print_python_style_block(printer: isl.printer, options: isl.ast_print_options, node: isl.ast_node_block):
  children = node.get_children()
  for i in range(children.size()):
    printer = children.get_at(i).print(printer, options)
  return printer

In [ ]:
with open('tmp/1.py', 'w') as f:
  ast_computation = lower_computation(s0_tensorized)
  printer = isl.printer.to_file(f)
  print_options = isl.ast_print_options.alloc()
  print_options = print_options.set_print_for(print_python_style_for)
  print_options = print_options.set_print_user(print_python_style_user)
  print_options = print_options.set_print_block(print_python_style_block)
  printer.set_output_format(isl.format.C)
  printer = ast_computation.print(printer, print_options)
  printer.flush()

with open('tmp/1.py', 'r') as f:
  print(f.read())

for ko in range(0, 7 + 1, 1):
    TransB = np.zeros([256,128])
    Trans(CommBroadcast(0),B[0:256,0:128],[ko],TransB[0:256,0:128],[x])
    TransA = np.zeros([64,256])
    Trans(CommBroadcast(1),A[0:64,0:256],[ko],TransA[0:64,0:256],[y])
    C[0:64,0:128] += TransA[0:64,0:256] @ TransB[0:256,0:128]



在codegen的过程中，通信操作我使用统一的api，包括这些api以及一些setup的代码，也需要生成出来。不过并不包含在compile的过程中。如果采用schedule tree进行代码生成，可以采用先插入mark节点后续callback处理的的方法正确生成setup相关代码，本教程为了简单起见，采用直接构造字符串写入文件的方式来处理：

In [ ]:
def codegen_setup(self: Computation, printer: isl.printer) -> isl.printer:
  printer.print_str('import numpy as np\n')
  printer.print_str('from mpi4py import MPI\n')
  printer.print_str('from enum import IntEnum\n')
  printer.print_str('import sys\n')
  printer.print_str('RANK = MPI.COMM_WORLD.Get_rank()\n')
  printer.print_str(f'MESH = [{",".join(map(str, self.mesh.shape))}]\n')
  printer.print_str(f'COMM_ALL = MPI.COMM_WORLD.Create_cart(MESH)\n')
  printer.print_str(f"({','.join(map(str, self.mesh.dims))}) = PIDS = COMM_ALL.Get_coords(RANK)\n")

  printer.print_str(f"""

class CommBroadcast:
  def __init__(self, *axes: int):
    self.axes: tuple[int] = axes

  def __call__(self, srcbuf: np.ndarray, source: list[int], recvbuf: np.ndarray, dest: list[int]):
    comm = COMM_ALL.Sub([True if i in self.axes else False for i in range(len(PIDS))])
    src_rank = comm.Get_cart_rank(source)
    if comm.Get_rank() == src_rank:
      np.copyto(recvbuf, srcbuf)
    comm.Bcast(recvbuf, root=src_rank)

class CommShift:
  def __init__(self, axes: tuple[int], delta: int):
    self.axes: tuple[int] = axes
    self.delta: int = delta

  def __call__(self, srcbuf: np.ndarray, source: list[int], recvbuf: np.ndarray, dest: list[int]):
    diffs = [p[0] - p[1] for p in zip(source, dest)]
    if not all([diffs[axis] == 0 for axis in self.axes]):
      src_rank = COMM_ALL.Get_cart_rank(
          [(cord - self.delta) % MESH[i] if i in self.axes else cord for i, cord in enumerate(dest)])
      dest_rank = COMM_ALL.Get_cart_rank(
          [(cord + self.delta) % MESH[i] if i in self.axes else cord for i, cord in enumerate(dest)])
      COMM_ALL.Sendrecv_replace(srcbuf, dest=dest_rank, source=src_rank)
    np.copyto(recvbuf, srcbuf)


class CommSendrecv:
  def __call__(self, srcbuf: np.ndarray, source: list[int], recvbuf: np.ndarray, dest: list[int]):
    COMM_ALL.Sendrecv(srcbuf, COMM_ALL.Get_cart_rank(dest), 0,
                      recvbuf, COMM_ALL.Get_cart_rank(source))

def Trans(comm, srcbuf: np.ndarray, source: list[int], recvbuf: np.ndarray, dest: list[int]):
  comm(srcbuf, source, recvbuf, dest)

""")
  return printer


def codegen_computation(self: Computation, printer: isl.printer, ast_computation: isl.ast_node):
  printer.print_str(f"def computation({', '.join([a.buffer.name for a in self.accesses])}):\n")
  printer.set_indent(4)
  printer = ast_computation.print(printer, print_options)
  printer.set_indent(-4)
  printer.end_line()
  return printer


def codegen_main(self: Computation, printer: isl.printer, ast_inputs: List[isl.ast_node], ast_outputs: List[isl.ast_node]):
  printer.print_str('if __name__ == "__main__":\n')
  for i, access in enumerate(self.accesses):
    printer.print_str(
        f'    Global{access.buffer.name} = np.load(sys.argv[{i + 1}], mmap_mode="r")\n')

  printer.set_indent(4)
  for in_ast in ast_inputs:
    printer = in_ast[0].print(printer, print_options)
    printer = in_ast[1].print(printer, print_options)
  for out_ast in ast_outputs:
    printer = out_ast[0].print(printer, print_options)
  printer.print_str(f"    computation({', '.join([a.buffer.name for a in self.accesses])})\n")
  for out_ast in ast_outputs:
    printer = out_ast[1].print(printer, print_options)
  printer.set_indent(-4)
  printer.end_line()
  return printer


def codegen_full(self: Computation, f: FileIO, ast_inputs: List[isl.ast_node], ast_outputs: List[Tuple[isl.ast_node, isl.ast_node]], ast_computation: isl.ast_node):
  printer = isl.printer.to_file(f)
  print_options = isl.ast_print_options.alloc()
  print_options = print_options.set_print_for(print_python_style_for)
  print_options = print_options.set_print_user(print_python_style_user)
  print_options = print_options.set_print_block(print_python_style_block)
  printer.set_output_format(isl.format.C)

  printer = codegen_setup(self, printer)
  printer = codegen_computation(self, printer, ast_computation)
  printer = codegen_main(self, printer, ast_inputs, ast_outputs)

  printer.flush()


def lower_and_codegen(self: Computation, f: FileIO):
  ast_shard_inputs = [lower_shard(self, access)
                      for access in self.accesses if access.buffer.usage == UsageKind.Input]
  ast_shard_outputs = [lower_shard(self, access)
                       for access in self.accesses if access.buffer.usage == UsageKind.Output]
  ast_computation = lower_computation(self)
  codegen_full(self, f, ast_shard_inputs, ast_shard_outputs, ast_computation)

In [ ]:
with open("tmp/1.py", "w") as f:
  codegen_full(s0_tensorized, f, [ast_shard_buffer_0, ast_shard_buffer_1], [ast_shard_buffer_2], ast_computation)

with open('tmp/1.py', 'r') as f:
  print(f.read())

import numpy as np
from mpi4py import MPI
from enum import IntEnum
import sys
RANK = MPI.COMM_WORLD.Get_rank()
MESH = [8,8]
COMM_ALL = MPI.COMM_WORLD.Create_cart(MESH)
(x,y) = PIDS = COMM_ALL.Get_coords(RANK)


class CommBroadcast:
  def __init__(self, *axes: int):
    self.axes: tuple[int] = axes

  def __call__(self, srcbuf: np.ndarray, source: list[int], recvbuf: np.ndarray, dest: list[int]):
    comm = COMM_ALL.Sub([True if i in self.axes else False for i in range(len(PIDS))])
    src_rank = comm.Get_cart_rank(source)
    if comm.Get_rank() == src_rank:
      np.copyto(recvbuf, srcbuf)
    comm.Bcast(recvbuf, root=src_rank)

class CommShift:
  def __init__(self, axes: tuple[int], delta: int):
    self.axes: tuple[int] = axes
    self.delta: int = delta

  def __call__(self, srcbuf: np.ndarray, source: list[int], recvbuf: np.ndarray, dest: list[int]):
    diffs = [p[0] - p[1] for p in zip(source, dest)]
    if not all([diffs[axis] == 0 for axis in self.axes]):
      src_rank = COMM_

开始创建输入并执行程序：

In [ ]:
import numpy as np
arrA = np.random.rand(512, 2048).astype(np.float32)
arrB = np.random.rand(2048, 1024).astype(np.float32)
np.save("tmp/arrA.npy", arrA)
np.save("tmp/arrB.npy", arrB)
np.save("tmp/arrC.npy", arrA @ arrB)


In [ ]:
import numpy as np
arrA = np.arange(16 * 64).astype(np.float32).reshape([16, 64])
arrB = np.arange(64 * 32).astype(np.float32).reshape([64, 32])
# arrB = np.ones_like(arrB)
np.save("tmp/arrA.npy", arrA)
np.save("tmp/arrB.npy", arrB)
np.save("tmp/arrC.npy", arrA @ arrB)


In [ ]:
# !mpirun --map-by :OVERSUBSCRIBE -n 64 python tmp/1.py tmp/arrA.npy tmp/arrB.npy tmp/arrC.npy 

In [ ]:
s0_tensorized

Computation(op=<OpKind.MatMul: 131>, domain=isl.set("{ s0[k, m, n] : 0 <= k <= 2047 and 0 <= m <= 511 and 0 <= n <= 1023 }"), schedule=isl.map("{ s0[k, m, n] -> s0[mo, no, ko, mi = m - 64mo, ni = n - 128no, ki = k - 256ko] : 0 <= k <= 2047 and 0 <= m <= 511 and 0 <= n <= 1023 and -63 + m <= 64mo <= m and -127 + n <= 128no <= n and -255 + k <= 256ko <= k }"), accesses=(Access(buffer=Buffer(name='C', dims=(IterVar(name='m', lower_bound=0, upper_bound=512, step=1), IterVar(name='n', lower_bound=0, upper_bound=1024, step=1)), dtype=Float32Type(), sharding=isl.map("{ C[m, n] -> C[x, y, m - 64x, local_n = n - 128y] : 0 <= m <= 511 and 0 <= n <= 1023 and 0 <= x <= 7 and -63 + m <= 64x <= m and 0 <= y <= 7 and -127 + n <= 128y <= n }"), usage=<UsageKind.Output: 1>), relation=isl.map("{ s0[k, m, n] -> C[m, n] }")), Access(buffer=Buffer(name='A', dims=(IterVar(name='m', lower_bound=0, upper_bound=512, step=1), IterVar(name='k', lower_bound=0, upper_bound=2048, step=1)), dtype=Float32Type(), shar

In [ ]:
x, y = IterVar.range('x', 8), IterVar.range('y', 8)
mo, no, mi, ni = IterVar.symbol('mo no mi ni')
ko, ki = IterVar.symbol('ko ki')
s0_distributed = s0.distribute([m, n], [mo, no], [mi, ni], Mesh((x, y))). \
    divide(k, ko, ki, x.extent). \
    reorder(mo, no, ko, mi, ni, ki)

s0_sharded = s0_distributed.shard('A', m @ x, k @ y). \
    shard('B', k @ x, n @ y). \
    shard('C', m @ x, n @ y)

s0_communicated = s0_sharded.communicate('A', ko, [no]).communicate('B', ko)

s0_tensorized = s0_communicated.tensorize([mi, ni, ki], OpKind.MatMul)

AttributeError: 'Computation' object has no attribute 'tensorize'

In [29]:
with open("tmp/1.py", "w") as f:
  lower_and_codegen(s0_tensorized, f)

with open('tmp/1.py', 'r') as f:
  print(f.read())

import numpy as np
from mpi4py import MPI
from enum import IntEnum
import sys
RANK = MPI.COMM_WORLD.Get_rank()
MESH = [8,8]
COMM_ALL = MPI.COMM_WORLD.Create_cart(MESH)
(x,y) = PIDS = COMM_ALL.Get_coords(RANK)


class CommBroadcast:
  def __init__(self, *axes: int):
    self.axes: tuple[int] = axes

  def __call__(self, srcbuf: np.ndarray, source: list[int], recvbuf: np.ndarray, dest: list[int]):
    comm = COMM_ALL.Sub([True if i in self.axes else False for i in range(len(PIDS))])
    src_rank = comm.Get_cart_rank(source)
    if comm.Get_rank() == src_rank:
      np.copyto(recvbuf, srcbuf)
    comm.Bcast(recvbuf, root=src_rank)

class CommShift:
  def __init__(self, axes: tuple[int], delta: int):
    self.axes: tuple[int] = axes
    self.delta: int = delta

  def __call__(self, srcbuf: np.ndarray, source: list[int], recvbuf: np.ndarray, dest: list[int]):
    diffs = [p[0] - p[1] for p in zip(source, dest)]
    if not all([diffs[axis] == 0 for axis in self.axes]):
      src_rank = COMM_